# Word2GM Training Data Pipeline

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare a Google 5gram corpora for Word2GM skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` NVMe storage
2. **Processing**: TensorFlow-native filtering, vocabulary building, and triplet generation
3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [1]:
# Set project root directory and add `src` to path
import sys
from pathlib import Path

PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the notebook setup utilities
from word2gm_fast.utils.notebook_setup import setup_data_preprocessing_notebook, enable_autoreload

# Enable autoreload for development
enable_autoreload()

# Set up environment (CPU-only for data preprocessing)
env = setup_data_preprocessing_notebook(project_root=PROJECT_ROOT)

# Extract commonly used modules for convenience
tf = env['tensorflow']
np = env['numpy']
pd = env['pandas']
batch_prepare_training_data = env['batch_prepare_training_data']
print_resource_summary = env['print_resource_summary']

<pre>Autoreload enabled</pre>

<pre>Project root: /scratch/edk202/word2gm-fast
TensorFlow version: 2.19.0
Device mode: CPU-only</pre>

<pre>Data preprocessing environment ready!</pre>

## Print Resource Summary

In [2]:
print_resource_summary()

<pre>SYSTEM RESOURCE SUMMARY
============================================================
Hostname: cm032.hpc.nyu.edu

Job Allocation:
   CPUs: 4
   Memory: 15.6 GB
   Requested partitions: short
   Running on: SSH failed: Host key verification failed.
   Job ID: 63398538
   Node list: cm032

GPU Information:
   Error: NVML Shared Library Not Found

TensorFlow GPU Detection:
   TensorFlow detects 0 GPU(s)
   Built with CUDA: True
============================================================</pre>

## Prepare Corpora

Here, we run the data-preparation pipeline from start to finish — reading preprocessed ngram corpora, generating all valid triplets, extracting the vocabulary, and saving the triplets and vocabulary as `tfrecord` files.

### Options for Data Preparation

You can control which years are processed and how the batch preparation runs by adjusting the arguments to `batch_prepare_training_data`:

**Ways to specify years:**
- `year_range="2010"` — Process a single year (e.g., only 2010).
- `year_range="2010,2012,2015"` — Process a comma-separated list of years.
- `year_range="2010-2015"` — Process a range of years, inclusive (2010 through 2015).
- `year_range="2010,2012-2014,2016"` — Combine individual years and ranges (2010, 2012, 2013, 2014, 2016).

**Other options:**
- `compress` — If `True`, output TFRecords are gzip-compressed. If `False`, output is uncompressed.
- `show_progress` — If `True`, display a progress bar for each year.
- `show_summary` — If `True`, print a summary of the processed data for each year.
- `use_multiprocessing` — If `True`, process years in parallel using multiple CPU cores (recommended for large datasets).

See the function docstring or source for more advanced options.

In [3]:
# Configuration
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

downsample_threshold = 1e-3  # Typical values: 1e-3 (aggressive), 1e-4, 1e-5 (conservative)

# Process year(s)
results = batch_prepare_training_data(
    corpus_dir=corpus_dir,
    year_range="1400-1600",
    compress=False,
    show_progress=True,
    show_summary=True,
    use_multiprocessing=True,
    downsample_threshold=downsample_threshold
)

Requested 201 years, found 10 corpus files.
Skipping 191 missing years (showing first 5): 1400, 1401, 1402, 1403, 1404, ...


PARALLEL BATCH PROCESSING
Processing 10 years
Using 4 parallel workers
Estimated speedup: 4.0x


1578 failed (1/10): Attempt to convert a value ({'UNK': 0, 'eye': 3, 'fix': 3, 'fleet': 2, 'gentry': 1, 'man': 1, 'mitre': 1, 'near': 1, 'nobility': 1, 'quality': 1, 'street': 1, 'upon': 3}) with an unsupported type (<class 'dict'>) to a Tensor.
1579 failed (2/10): Attempt to convert a value ({'UNK': 0, 'acid': 2, 'authorize': 2, 'book': 1, 'buy': 2, 'college': 2, 'facsimile': 2, 'free': 2, 'fund': 1, 'harvard': 1, 'library': 2, 'original': 1, 'paper': 1, 'richmond': 1, 'sheldon': 1, 'xerography': 1}) with an unsupported type (<class 'dict'>) to a Tensor.
1583 failed (3/10): Attempt to convert a value ({'UNK': 0, 'abuse': 1, 'addition': 1, 'anatomy': 1, 'august': 2, 'character': 1, 'circumstance': 2, 'complete': 1, 'conclusion': 1, 'edition': 4, 'elizabeth': 1, 'fifth': 1, 'form': 1, 'make': 1, 'may': 2, 'middle': 2, 'number': 1, 'original': 2, 'page': 1, 'presume': 2, 'proper': 1, 'quite': 1, 'reign': 1, 'several': 1, 'think': 1, 'title': 1, 'various': 2}) with an unsupported type (<c